# Block Sorting Algorithm
This optimization is modeled after the *multiple knapsacks problem*, in which the goal is to pack a set of items, with given values and sizes into a container with a maximum capacity. In this implementation, the goal is to assign blocks of leads to the Cadwell amplifiers (A-D, 9 rows each).

---
> Author:    Justin Campbell    
> Contact:   justin.campbell@hsc.utah.edu   
> Version:   04/10/2024  

In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp

### Define Block Map and Amplifier Constraints

In [ ]:
# Filepath for block map (if exists)
block_map_file = '/Users/justincampbell/Desktop/BlockMap.csv'

In [ ]:
# Load if block map file exists, otherwise specify manually
try:
    block_map = pd.read_csv(block_map_file)
    block_sizes = block_map['Size'].tolist()
    block_values = block_map['Value'].tolist()
    block_labels = block_map['Label'].tolist()

except:
    block_sizes = [
        5, 5, 4, 4, 
        5, 5, 5, 4, 
        4, 4, 5, 5, 
        5, 2, 2, 2]

    block_values = [ 
        1, 1, 1, 1, 
        1, 1, 1, 1, 
        1, 1, 1, 1, 
        1, 1, 1, 1]

    block_labels = [
        'LCM', 'LAINS', 'LPHIP', 'LAHIP', 
        'LPINS', 'RAINS', 'RPINS', 'RPHIP', 
        'RAHIP', 'RAMY', 'RACC', 'ROFC', 
        'ANT', 'LACC', 'LOFC', 'LAMY']
    
    block_map = pd.DataFrame({'Label': block_labels, 'Size': block_sizes, 'Value': block_values})

# Display
block_map

In [ ]:
# Amplifier constraints
amp_labels = ['A', 'B', 'C', 'D']
amp_size = 9
available_amps = 4 # modify if D is occupied by microelectrodes

### Setup Solver

In [ ]:
# Create variables
data = {}
data["sizes"] = block_sizes
data["values"] = block_values
data["num_blocks"] = len(data["sizes"])
data["all_blocks"] = range(data["num_blocks"])
data["amp_capacity"] = [amp_size] * available_amps
data["num_amps"] = len(data["amp_capacity"])
data["all_amps"] = range(data["num_amps"])

In [ ]:
solver = pywraplp.Solver.CreateSolver("SCIP")

x = {}
for i in data["all_blocks"]:
    for b in data["all_amps"]:
        x[i, b] = solver.BoolVar(f"x_{i}_{b}")

# Each block is assigned to at most one amplifier
for i in data["all_blocks"]:
    solver.Add(sum(x[i, b] for b in data["all_amps"]) <= 1)

# The number of blocks cannot exceed the amplifier capacity
for b in data["all_amps"]:
    solver.Add(
        sum(x[i, b] * data["sizes"][i] for i in data["all_blocks"])
        <= data["amp_capacity"][b])

In [ ]:
# Maximize total value of blocks in amplifiers
objective = solver.Objective()
for i in data["all_blocks"]:
    for b in data["all_amps"]:
        objective.SetCoefficient(x[i, b], data["values"][i])
        
objective.SetMaximization()

### Find Optimal Solution

In [ ]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Suggested Order (Total Combined Value = %.1f):\n' %objective.Value())
    
    total_weight = 0
    for b in range(available_amps):
        print('Amp %s' %amp_labels[b])
        bin_weight = 0
        bin_value = 0
        for i in range(len(block_labels)):
            if x[i, b].solution_value() > 0:
                print('%s (Size: %i, Value: %.1f)' % (block_labels[i], data["sizes"][i], data["values"][i]))
                bin_weight += data["sizes"][i]
                bin_value += data["values"][i]
        print('Blocks Used: %i/%i\n' %(bin_weight, data["amp_capacity"][b]))
        total_weight += bin_weight

else:
    print('The problem does not have an optimal solution.')

In [ ]:
# Display blocks that are not assigned in map
print('Unused Blocks:\n')
for i in range(len(block_labels)):
    used = False
    for b in range(available_amps):
        if x[i, b].solution_value() > 0:
            used = True
            break
    if not used:
        print('%s (Size: %i, Value: %.1f)' % (block_labels[i], data["sizes"][i], data["values"][i]))